In [ ]:
# Iniciar sesion de Google Earth Engine
# Aca deben poner su ID de proyecto de GEE
# Por defecto estara como (project='my-project')

import ee
import geemap

ee.Authenticate()
ee.Initialize(project='ee-joaquin-nuevo')
print("🔑 Autenticación completada e inicialización exitosa.")

In [ ]:
asset_id = 'projects/ee-joaquin-nuevo/assets/Accopata'

try:
    # Intentar cargar el asset
    mi_poligono = ee.FeatureCollection(asset_id)

    # Obtener información básica para confirmar que existe
    info = mi_poligono.first().getInfo()
    print("✅ ¡Conexión exitosa!")
    print("Información del primer elemento:", info['properties'])

except Exception as e:
    print("❌ Error: No se pudo encontrar el asset. Verifica que el ID sea correcto.")
    print(e)

In [ ]:
import ee
import geemap
import ipywidgets as widgets
from IPython.display import display

# 1. Tu Asset ID y carga de ROI (ya verificado)
asset_id = 'projects/ee-joaquin-nuevo/assets/Accopata'
roi = ee.FeatureCollection(asset_id)

# 2. Procesamiento de la imagen (el código que compartiste)
image = (ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
    .filterBounds(roi)
    .filterDate('2023-01-01', '2023-12-31')
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20))
    .median()
    .clip(roi))

# 3. PARÁMETROS PARA IMAGEN NORMAL (Color Real)
# Usamos B4, B3, B2. El valor 'max' de 3000 y 'gamma' de 1.4
# suelen dar la mejor apariencia visual.
vis_params_sharp = {
    'bands': ['B4', 'B3', 'B2'],
    'min': 0,
    'max': 2500, # Bajamos el máximo para ganar contraste
    'gamma': 1.6   # Subimos el gamma para saturar un poco más los colores
}
# 4. Configurar el Mapa con fondo satelital de Google
Map = geemap.Map(basemap='SATELLITE')
Map.centerObject(roi, 14)

# 5. Agregar las capas
Map.addLayer(image, vis_params_true_color, 'Sentinel-2 Color Real')
Map.addLayer(roi, {'color': 'red'}, 'Límite Accopata', False) # El límite en rojo (apagado por defecto)

# Mostrar el mapa
Map

In [ ]:
# 1. Tu Asset ID (el que verificamos recién)
asset_id = 'projects/ee-joaquin-nuevo/assets/Accopata' # Ajusta la ruta exacta aquí

# 2. Cargar tu polígono
roi = ee.FeatureCollection(asset_id)

# 3. Filtrar Sentinel-2 para la zona de Accopata
# Usamos un filtro de nubes estricto porque en zonas altas de Perú hay mucha nube
image = (ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
    .filterBounds(roi)
    .filterDate('2023-01-01', '2023-12-31')
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20))
    .median()
    .clip(roi)) # <--- Aquí cortamos la imagen con tu polígono

# 4. Calcular un Índice de Vegetación (NDVI) rápido para ver la salud
ndvi = image.normalizedDifference(['B8', 'B4']).rename('NDVI')


In [ ]:
import ee
import geemap
import ipywidgets as widgets
import matplotlib.pyplot as plt
from IPython.display import display

# 1. Función para calcular el NDVI promedio en el polígono
def get_ndvi_avg(img):
    avg = img.normalizedDifference(['B8', 'B4']).reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=roi,
        scale=30
    ).get('nd')
    return img.set('date', img.date().format('YYYY-MM-dd')).set('avg_ndvi', avg)

# 2. Extraer los datos (esto puede tardar unos segundos)
print("Extrayendo datos de Sentinel-2 para Accopata...")
col_ndvi = (ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
            .filterBounds(roi)
            .filterDate('2023-01-01', '2023-12-31')
            .map(get_ndvi_avg))

# Convertir la información de GEE a listas de Python
dates = col_ndvi.aggregate_array('date').getInfo()
values = col_ndvi.aggregate_array('avg_ndvi').getInfo()

# 3. Configurar el Mapa
Map = geemap.Map()
Map.centerObject(roi, 14)
vis_ndvi = {'min': 0, 'max': 0.8, 'palette': ['white', 'yellow', 'green', 'darkgreen']}
Map.addLayer(ndvi, vis_ndvi, 'NDVI Actual')
Map.addLayer(roi, {'color': 'red'}, 'Límite Accopata', False)

# 4. Crear el Dashboard (Mapa arriba, Gráfica abajo)
output_widget = widgets.Output(layout={'border': '1px solid black', 'margin': '10px 0'})

with output_widget:
    plt.figure(figsize=(10, 4))
    plt.plot(dates, values, marker='o', linestyle='-', color='forestgreen', markersize=4)
    plt.title('Evolución de la Vegetación en Accopata (2023)')
    plt.ylabel('NDVI Promedio')
    plt.xlabel('Fecha')
    plt.xticks(rotation=45)
    plt.grid(True, alpha=0.3)
    plt.show()

# Mostrar el resultado final
display(widgets.VBox([Map, output_widget]))

In [ ]:
# Salir de cualquier carpeta y volver a la raíz de Colab
%cd /content/

# Borrar la carpeta que está dando problemas para empezar de cero
!rm -rf Google-Earth-Engine-Proyectos

In [ ]:

# 1. Configurar identidad (por si acaso)
!git config --global user.email "joaquin.espinoza@utec.edu.pe"
!git config --global user.name "Joako1202"

# 2. Inicializar Git en la carpeta actual
!git init

# 3. Vincular con tu repositorio de GitHub
!git remote add origin https://github.com/Joako1202/Google-Earth-Engine-Proyectos.git

# 4. Crear el archivo README y añadir lo que tengas
!echo "# Google Earth Engine Proyectos" > README.md
# Si tienes tu archivo .ipynb en /content/, muévelo aquí:
# !cp /content/TuArchivo.ipynb .

# 5. Agregar, Guardar y Renombrar rama
!git add .
!git commit -m "Primer envío limpio de Accopata"
!git branch -M main

# 6. SUBIR (Reemplaza TU_TOKEN por tu token de GitHub)
!git push https://github.com/Joako1202/Google-Earth-Engine-Proyectos.git



In [ ]:

# 1. Tu Token
TOKEN = "TOKEN_ELIMINADO"

# 2. Configurar la URL de forma limpia
!git remote set-url origin https://{TOKEN}@github.com/Joako1202/Google-Earth-Engine-Proyectos.git

# 3. Asegurar que la rama se llame main
!git branch -M main

# 4. Empujar los cambios (el comando definitivo)
!git push -u origin main

In [ ]:
# 1. Listar los archivos para encontrar tu notebook
!ls /content/

# 2. Copia el archivo a la carpeta actual (reemplaza 'TuNombre.ipynb' por el real)
# Si no sabes el nombre, este comando copiará TODOS los archivos .ipynb que encuentre
!cp /content/*.ipynb .

In [ ]:
import json
import os

# Este comando guarda el notebook actual en un archivo llamado GEE_01.ipynb
# Nota: Si te pide permiso para acceder a Drive, dáselo.
from google.colab import drive
!pip install colab-convert &> /dev/null

# Intentamos guardarlo directamente como archivo
!cp "/content/drive/MyDrive/Colab Notebooks/NOMBRE_DE_TU_NOTEBOOK.ipynb" "/content/GEE_01.ipynb"